In [1]:
import math

import rasterio
import rasterio.plot
from rasterio.mask import mask 

from osgeo import gdal
gdal.UseExceptions()
    
import geopandas as gpd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from rasterio.plot import plotting_extent
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

### Abrir arquivos

- Ortomosaico
- Grade
- Vetor das copas

In [2]:
img = rasterio.open('../IMGS/odm_orthophoto.tif')

grid = gpd.read_file('../VECTOR/grade_completa.geojson').set_index('id')
grid_2 = gpd.read_file('../VECTOR/grade_talhao2.geojson').set_index('id')

copas = gpd.read_file('../VECTOR/copas.geojson').set_index('id').sort_index()

### Cortar ortomosaico

In [9]:
# Melhorar a função do corte
with rasterio.open('../IMGS/odm_orthophoto.tif') as img:
#     for i in grid.index:
    for i in [437]:
        
        dic = grid.loc[[i]].__geo_interface__
        
        polygon = [dic['features'][0][ 'geometry']]
        
        out_image, out_transform = mask(img, polygon, crop=True)
        
        out_meta = img.meta.copy()
        
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
        
        with rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(i), "w", **out_meta) as dest:
            dest.write(out_image)
            
            print(dest.width)

203


### Cortar vetor das copas

In [ ]:
list_empty = []

for i in grid.index:
    corte = gpd.clip(copas, grid.loc[[i]])
    
    if corte.empty:
        list_empty.append(i)
    else:
        corte.to_file('../VECTOR/CORTES/corte_{}.geojson'.format(i), driver="GeoJSON")
        
print('Grids não salvas: ',list_empty)

### Plotar

In [ ]:
fig, ax = plt.subplots(5, 5, figsize=(15,15))

for i, axs in zip(grid_2.index,ax.flat):
    
    img = rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(i))
    vetor = gpd.read_file('../VECTOR/CORTES/corte_{}.geojson'.format(i))

    plot_extent = plotting_extent(img)
    ep.plot_rgb(img.read(),
               rgb = [0, 1, 2],
               extent = plot_extent,
               ax=axs)

    vetor.plot(ax=axs, facecolor='none', edgecolor='red')
    
    axs.set_title('Tile {}'.format(i))

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
id_grid = list(grid_2.index)

num = 22

img = rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(id_grid[num]))
vetor = gpd.read_file('../VECTOR/CORTES/corte_{}.geojson'.format(id_grid[num]))

plot_extent = plotting_extent(img)
ep.plot_rgb(img.read(),
           rgb = [0, 1, 2],
           extent = plot_extent,
           ax=ax)

vetor.plot(ax=ax, facecolor='none', edgecolor='red');

In [ ]:
ids = []
check_w = []
check_h = []
for z in grid_2.index:
    
    img = rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(z))
    
    if img.height not in check_h:
        check_h.append(img.height)
        
    if img.width not in check_w:
        check_w.append(img.width)
        ids.append(z)
        
print(check_h)
print('\n')
print(check_w, ids)

In [ ]:
id_grid_t2 = list(grid_2.index)

multi = {}
for z in id_grid_t2:
    
    img = rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(z))
    corte = gpd.read_file('../VECTOR/CORTES/corte_{}.geojson'.format(z))
    
    corte.insert(len(corte.columns),'geometry_image', 0)
    corte['geometry_image'] = corte['geometry_image'].astype('object')

    features = []
    for i in range(corte.shape[0]):
        
        if corte.loc[i,'geometry'].geom_type == 'MultiPolygon':
            features.append(i)
    
    if len(features) != 0:
        multi[z] = features

len(multi)

In [ ]:
for z in list(grid_2.index):
    
    img = rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(z))
    corte = gpd.read_file('../VECTOR/CORTES/corte_{}.geojson'.format(z))
    
    corte.insert(len(corte.columns)-1,'geometry_image', 0)
    corte['geometry_image'] = corte['geometry_image'].astype('string')

    for i in corte.index:
        
        # [FEITO] Preciso avaliar a ocorrencia de multipartes 
        # A função .coords e .exterior não se aplicam a multipartes [FEITO]
        coord = list(corte.loc[i,'geometry'].exterior.coords)
        
        segmentation = ''
        # Atenção aqui!! A visualização das annotations indicou que na lista de coors_img, o y precede o x
        for xy in coord:
            segmentation += str(round(img.index(xy[0],xy[1], float)[1],2)) + ', ' + str(round(img.index(xy[0],xy[1], float)[0],2)) + ', '
        
        
        segmentation = segmentation[:-2]
        corte.at[i,'geometry_image'] = segmentation
    
    corte.to_file('../VECTOR/CORTES/EDITADOS/corte_{}.geojson'.format(z), driver="GeoJSON")

In [ ]:
img = rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(211))
corte = gpd.read_file('../VECTOR/CORTES/EDITADOS/corte_{}.geojson'.format(211))


xy_img = [float(x) for x in corte.loc[0,'geometry_image'].split(', ')]

pairs = []

for i in range(0, len(xy_img)-1, 2):
    pairs.append((xy_img[i], xy_img[i+1]))


xy_coords = []
for i in xy_img:

    xy_coords.append(img.transform * i)
    
xy_coords

In [ ]:
xy_img

In [ ]:
img.index(xy[0],xy[1], float, precision=2)

In [ ]:
import math

float(3.2)

In [ ]:
gpd.read_file('../VECTOR/CORTES/EDITADOS/corte_{}.geojson'.format(38))

In [ ]:
corte = gpd.read_file('../VECTOR/CORTES/corte_{}.geojson'.format(id_grid[22]))

corte.insert(len(corte.columns),'geometry_image', 0)
corte['geometry_image'] = corte['geometry_image'].astype('object')

for i in range(corte.shape[0]):
    
    coord = list(corte.iloc[i,:]['geometry'].exterior.coords)
    
    segmentation = []
    
    for xy in coord:
        segmentation.append(img.index(xy[0],xy[1])[0])
        segmentation.append(img.index(xy[0],xy[1])[1])
        
    
    corte.iat[i,4] = str(segmentation)

corte

In [ ]:
info = {
        "description": "Felipe Sa 2021 - HLB",
        "url": "http://siteaqui.com",
        "version": "1.0",
        "year": 2021,
        "contributor": "Felipe Sa",
        "date_created": "2021/01/01"
        }

licenses = [
                {
                "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
                "id": 1,
                "name": "Attribution-NonCommercial-ShareAlike License"
                },
                {
                "url": "http://creativecommons.org/licenses/by-nc/2.0/",
                "id": 2,
                "name": "Attribution-NonCommercial License"
                }
            ]

In [ ]:
annotation = {
                "info": info,
                "licenses": licenses,
                "images": [],
                "annotations": [],
                "categories": [],   # <-- Not in Captions annotations
                "segment_info": []  # <-- Only in Panoptic annotations
                }

In [ ]:
licenses

In [ ]:
annotation

In [ ]:
info = {
        "description": "Felipe Sa 2021 - HLB",
        "url": "http://siteaqui.com",
        "version": "1.0",
        "year": 2021,
        "contributor": "Felipe Sa",
        "date_created": "2021/01/01"
        }